In [16]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from pathlib import Path
from PIL import Image
from sklearn.metrics import precision_score, recall_score, f1_score, jaccard_score

In [25]:
import torch
import torch.nn as nn

class UNet(nn.Module):
    def __init__(self):
        super(UNet, self).__init__()
        
        # Encoder 部分（下采样）
        self.encoder1 = self.conv_block(3, 64)
        self.encoder2 = self.conv_block(64, 128)
        self.encoder3 = self.conv_block(128, 256)
        self.encoder4 = self.conv_block(256, 512)
        
        # Bottom 部分（最底层）
        self.bottom = self.conv_block(512, 1024)
        
        # Decoder 部分（上采样）
        self.upconv4 = nn.ConvTranspose2d(1024, 512, kernel_size=2, stride=2)
        self.decoder4 = self.conv_block(1024, 512)
        
        self.upconv3 = nn.ConvTranspose2d(512, 256, kernel_size=2, stride=2)
        self.decoder3 = self.conv_block(512, 256)
        
        self.upconv2 = nn.ConvTranspose2d(256, 128, kernel_size=2, stride=2)
        self.decoder2 = self.conv_block(256, 128)
        
        self.upconv1 = nn.ConvTranspose2d(128, 64, kernel_size=2, stride=2)
        self.decoder1 = self.conv_block(128, 64)
        
        # 最后一层，用于输出分割结果
        self.output_conv = nn.Conv2d(64, 1, kernel_size=1)
        
    def conv_block(self, in_channels, out_channels):
        """卷积块：包含两层卷积，每层卷积后跟一个 BatchNorm 和 ReLU 激活"""
        return nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        )
    
    def forward(self, x):
        # 编码器部分
        enc1 = self.encoder1(x)
        enc2 = self.encoder2(nn.MaxPool2d(2)(enc1))
        enc3 = self.encoder3(nn.MaxPool2d(2)(enc2))
        enc4 = self.encoder4(nn.MaxPool2d(2)(enc3))
        
        # Bottom
        bottom = self.bottom(nn.MaxPool2d(2)(enc4))
        
        # 解码器部分 + 跳跃连接
        dec4 = self.upconv4(bottom)
        dec4 = torch.cat((dec4, enc4), dim=1)  # 跳跃连接
        dec4 = self.decoder4(dec4)
        
        dec3 = self.upconv3(dec4)
        dec3 = torch.cat((dec3, enc3), dim=1)
        dec3 = self.decoder3(dec3)
        
        dec2 = self.upconv2(dec3)
        dec2 = torch.cat((dec2, enc2), dim=1)
        dec2 = self.decoder2(dec2)
        
        dec1 = self.upconv1(dec2)
        dec1 = torch.cat((dec1, enc1), dim=1)
        dec1 = self.decoder1(dec1)
        
        # 输出层
        output = self.output_conv(dec1)
        output = torch.sigmoid(output)  # 将输出映射到 [0,1] 区间
        return output


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from pathlib import Path
from PIL import Image
from sklearn.metrics import precision_score, recall_score, f1_score, jaccard_score

# 自定义数据集类
class ContrailDataset(Dataset):
    def __init__(self, image_dir, mask_dir, transform=None):
        super().__init__()
        self.image_dir = Path(image_dir)
        self.mask_dir = Path(mask_dir)
        self.files = [f for f in self.image_dir.iterdir() if f.is_file() and f.suffix in ['.png', '.jpg', '.jpeg']]
        self.transform = transform

    def __len__(self):
        return len(self.files)

    def __getitem__(self, idx):
        image_path = self.files[idx]
        mask_path = self.mask_dir / image_path.name

        image = Image.open(image_path).convert('RGB')  # 读取为RGB图像
        mask = Image.open(mask_path).convert('L')  # 读取为灰度图像

        if self.transform:
            image = self.transform(image)
            mask = transforms.ToTensor()(mask)

        mask = (mask > 0).float()  # 将掩码二值化
        return image, mask.unsqueeze(0)  # 在channel维度增加一个维度

# 定义Dice损失函数
class DiceLoss(nn.Module):
    def forward(self, inputs, targets, smooth=1):
        inputs = torch.sigmoid(inputs)
        inputs = inputs.view(-1)
        targets = targets.view(-1)
        intersection = (inputs * targets).sum()
        dice = (2. * intersection + smooth) / (inputs.sum() + targets.sum() + smooth)
        return 1 - dice

# 计算 Precision, Recall, F1 Score, IoU
def calculate_metrics(preds, labels):
    preds = (torch.sigmoid(preds) > 0.5).cpu().numpy().flatten()  # 二值化预测
    labels = labels.cpu().numpy().flatten()

    precision = precision_score(labels, preds, zero_division=1)
    recall = recall_score(labels, preds, zero_division=1)
    f1 = f1_score(labels, preds, zero_division=1)
    iou = jaccard_score(labels, preds, zero_division=1)

    return precision, recall, f1, iou

# 计算平均 IoU（mIoU），对于二分类问题，mIoU 就是 IoU 本身
def calculate_mIoU(preds, labels):
    preds = (torch.sigmoid(preds) > 0.5).cpu().numpy()
    labels = labels.cpu().numpy()

    ious = []
    for i in range(preds.shape[0]):  # 针对每一张图片
        iou = jaccard_score(labels[i].flatten(), preds[i].flatten(), zero_division=1)
        ious.append(iou)
    
    return sum(ious) / len(ious)

def calculate_accuracy(preds, labels):
    # 使用Sigmoid并将输出二值化
    preds = (torch.sigmoid(preds) > 0.5).float()  # 二值化预测
    # 确保预测和标签维度一致
    preds = preds.view(-1)
    labels = labels.view(-1)

    # 计算正确的预测像素总数
    correct = (preds == labels).sum().item()
    # 计算总像素数
    total = labels.size(0)

    # 返回准确率
    acc = correct / total
    return acc

# 训练函数
def train_model(model, train_loader, valid_loader, loss_fn, optimizer, num_epochs, device):
    best_f1 = 0

    for epoch in range(num_epochs):
        model.train()
        train_loss = 0
        train_acc = 0

        # Training phase
        for images, masks in train_loader:
            images, masks = images.to(device), masks.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = loss_fn(outputs, masks)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()
            acc = calculate_accuracy(outputs, masks)
            train_acc += acc

        # Validation phase
        model.eval()
        valid_loss = 0
        valid_acc = 0
        valid_precision = 0
        valid_recall = 0
        valid_f1 = 0
        valid_iou = 0
        valid_miou = 0

        with torch.no_grad():
            for images, masks in valid_loader:
                images, masks = images.to(device), masks.to(device)
                outputs = model(images)

                loss = loss_fn(outputs, masks)
                valid_loss += loss.item()

                acc = calculate_accuracy(outputs, masks)
                valid_acc += acc

                precision, recall, f1, iou = calculate_metrics(outputs, masks)
                valid_precision += precision
                valid_recall += recall
                valid_f1 += f1
                valid_iou += iou
                valid_miou += calculate_mIoU(outputs, masks)

        # 平均值
        avg_train_loss = train_loss / len(train_loader)
        avg_train_acc = train_acc / len(train_loader)
        avg_valid_loss = valid_loss / len(valid_loader)
        avg_valid_acc = valid_acc / len(valid_loader)
        avg_valid_precision = valid_precision / len(valid_loader)
        avg_valid_recall = valid_recall / len(valid_loader)
        avg_valid_f1 = valid_f1 / len(valid_loader)
        avg_valid_iou = valid_iou / len(valid_loader)
        avg_valid_miou = valid_miou / len(valid_loader)

        print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {avg_train_loss:.4f}, Train Acc: {avg_train_acc:.4f}, '
              f'Val Loss: {avg_valid_loss:.4f}, Val Acc: {avg_valid_acc:.4f}, '
              f'Precision: {avg_valid_precision:.4f}, Recall: {avg_valid_recall:.4f}, '
              f'F1 Score: {avg_valid_f1:.4f}, IoU: {avg_valid_iou:.4f}, mIoU: {avg_valid_miou:.4f}')

        # 保存最好的模型
        if avg_valid_f1 > best_f1:
            best_f1 = avg_valid_f1
            torch.save(model.state_dict(), 'UNet1_model.pth')

# 设置参数
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
image_dir = '/Users/camus/Desktop/trainX/image'
mask_dir = '/Users/camus/Desktop/trainX/maskimage'

batch_size = 4
num_epochs = 20

# 图像预处理，直接使用256x256的尺寸
transform = transforms.Compose([
    transforms.Resize((256, 256)),  # 将图像调整为256x256
    transforms.ToTensor(),
])

# 加载数据集
dataset = ContrailDataset(image_dir, mask_dir, transform=transform)
train_size = int(0.75 * len(dataset))
valid_size = len(dataset) - train_size
train_dataset, valid_dataset = torch.utils.data.random_split(dataset, [train_size, valid_size])

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=batch_size)

# 初始化模型和损失函数
model = UNet().to(device)
loss_fn = DiceLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# 训练模型
train_model(model, train_loader, valid_loader, loss_fn, optimizer, num_epochs, device)

Epoch [1/20], Train Loss: 0.7891, Train Acc: 0.1039, Val Loss: 0.8125, Val Acc: 0.0954, Precision: 0.0954, Recall: 1.0000, F1 Score: 0.1711, IoU: 0.0954, mIoU: 0.0954
Epoch [2/20], Train Loss: 0.7804, Train Acc: 0.1037, Val Loss: 0.7948, Val Acc: 0.0954, Precision: 0.0954, Recall: 1.0000, F1 Score: 0.1711, IoU: 0.0954, mIoU: 0.0954
Epoch [3/20], Train Loss: 0.7761, Train Acc: 0.1046, Val Loss: 0.8014, Val Acc: 0.0954, Precision: 0.0954, Recall: 1.0000, F1 Score: 0.1711, IoU: 0.0954, mIoU: 0.0954
Epoch [4/20], Train Loss: 0.7791, Train Acc: 0.1031, Val Loss: 0.7955, Val Acc: 0.0954, Precision: 0.0954, Recall: 1.0000, F1 Score: 0.1711, IoU: 0.0954, mIoU: 0.0954


In [4]:
# 定义完整的U-Net模型
class UNet(nn.Module):
    def __init__(self):
        super(UNet, self).__init__()
        
        # Contracting Path (Encoder)
        self.encoder1 = self.conv_block(3, 64)
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        
        self.encoder2 = self.conv_block(64, 128)
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        
        self.encoder3 = self.conv_block(128, 256)
        self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2)
        
        self.encoder4 = self.conv_block(256, 512)
        self.pool4 = nn.MaxPool2d(kernel_size=2, stride=2)
        
        # Bottleneck
        self.bottleneck = self.conv_block(512, 1024)
        
        # Expansive Path (Decoder)
        self.upconv4 = nn.ConvTranspose2d(1024, 512, kernel_size=2, stride=2)
        self.decoder4 = self.conv_block(1024, 512)
        
        self.upconv3 = nn.ConvTranspose2d(512, 256, kernel_size=2, stride=2)
        self.decoder3 = self.conv_block(512, 256)
        
        self.upconv2 = nn.ConvTranspose2d(256, 128, kernel_size=2, stride=2)
        self.decoder2 = self.conv_block(256, 128)
        
        self.upconv1 = nn.ConvTranspose2d(128, 64, kernel_size=2, stride=2)
        self.decoder1 = self.conv_block(128, 64)
        
        # Output Layer
        self.conv_final = nn.Conv2d(64, 1, kernel_size=1)

    def conv_block(self, in_channels, out_channels):
        return nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
        )

    def forward(self, x):
        # Contracting path (Encoder)
        e1 = self.encoder1(x)
        p1 = self.pool1(e1)
        
        e2 = self.encoder2(p1)
        p2 = self.pool2(e2)
        
        e3 = self.encoder3(p2)
        p3 = self.pool3(e3)
        
        e4 = self.encoder4(p3)
        p4 = self.pool4(e4)
        
        # Bottleneck
        b = self.bottleneck(p4)
        
        # Expansive path (Decoder)
        u4 = self.upconv4(b)
        d4 = torch.cat((u4, e4), dim=1)
        d4 = self.decoder4(d4)
        
        u3 = self.upconv3(d4)
        d3 = torch.cat((u3, e3), dim=1)
        d3 = self.decoder3(d3)
        
        u2 = self.upconv2(d3)
        d2 = torch.cat((u2, e2), dim=1)
        d2 = self.decoder2(d2)
        
        u1 = self.upconv1(d2)
        d1 = torch.cat((u1, e1), dim=1)
        d1 = self.decoder1(d1)
        
        # Output layer
        out = self.conv_final(d1)
        return out


# 设置参数
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
image_dir = '/Users/camus/Desktop/trainX2/image'
mask_dir = '/Users/camus/Desktop/trainX2/maskimage'

batch_size = 4
num_epochs = 42

# 图像预处理，直接使用256x256的尺寸
transform = transforms.Compose([
    transforms.Resize((256, 256)),  # 将图像调整为256x256
    transforms.ToTensor(),
])
# 加载数据集
dataset = ContrailDataset(image_dir, mask_dir, transform=transform)
train_size = int(0.75 * len(dataset))
valid_size = len(dataset) - train_size
train_dataset, valid_dataset = torch.utils.data.random_split(dataset, [train_size, valid_size])

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=batch_size)
# 继续模型训练

# 初始化模型和损失函数
model = UNet().to(device)
loss_fn = DiceLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# 继续训练前，加载模型的权重
checkpoint_path = 'UNet_model.pth'
if Path(checkpoint_path).exists():
    model.load_state_dict(torch.load(checkpoint_path))
    print("Model loaded from", checkpoint_path)
else:
    print("No checkpoint found, training from scratch.")

# 继续训练
train_model(model, train_loader, valid_loader, loss_fn, optimizer, num_epochs, device)

Model loaded from UNet_model.pth
Epoch [1/42], Train Loss: 0.8089, Train Acc: 0.1068, Val Loss: 0.8181, Val Acc: 0.1017, Precision: 0.1017, Recall: 1.0000, F1 Score: 0.1819, IoU: 0.1017, mIoU: 0.1017
Epoch [2/42], Train Loss: 0.8089, Train Acc: 0.1069, Val Loss: 0.8181, Val Acc: 0.1017, Precision: 0.1017, Recall: 1.0000, F1 Score: 0.1819, IoU: 0.1017, mIoU: 0.1017
Epoch [3/42], Train Loss: 0.8101, Train Acc: 0.1064, Val Loss: 0.8181, Val Acc: 0.1017, Precision: 0.1017, Recall: 1.0000, F1 Score: 0.1819, IoU: 0.1017, mIoU: 0.1017
Epoch [4/42], Train Loss: 0.8092, Train Acc: 0.1067, Val Loss: 0.8181, Val Acc: 0.1017, Precision: 0.1017, Recall: 1.0000, F1 Score: 0.1819, IoU: 0.1017, mIoU: 0.1017
Epoch [5/42], Train Loss: 0.8103, Train Acc: 0.1060, Val Loss: 0.8181, Val Acc: 0.1017, Precision: 0.1017, Recall: 1.0000, F1 Score: 0.1819, IoU: 0.1017, mIoU: 0.1017
Epoch [6/42], Train Loss: 0.8085, Train Acc: 0.1071, Val Loss: 0.8181, Val Acc: 0.1017, Precision: 0.1017, Recall: 1.0000, F1 Score: